In [1]:
import pandas as pd
from geopy.distance import geodesic
from IPython.display import clear_output

from Libreria.cargar_configbd import cargar_configbd
from Libreria.cargar_mes_a_procesar import cargar_mes_a_procesar

In [2]:
# Conectar a la base de datos PostgreSQL
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
# Crear un cursor para ejecutar consultas
cur = conn.cursor()

In [3]:
mes, año = cargar_mes_a_procesar.leer_csv_en_lista('mes_a_procesar.csv')
mes = 'abr' # De septiembre ya saque auto, bus y bici
año = 2024
print(f"{mes}{año}")

abr2024


In [4]:
# ---- CLASIFICAR MODO BICICLETA

In [4]:
def detectar_segmento_ciclovia(df_viaje, df_ciclovias, umbral_distancia, num_puntos):
    def buscar_punto_ciclovia_cercana(punto, ciclovias, umbral_distancia):
        if len(ciclovias) > 0:
            for idx, row in ciclovias.iterrows():
                distancia = geodesic((punto['latitude'], punto['longitude']), (row['latitud'], row['longitud'])).meters
                
                if distancia <= umbral_distancia:
                    return row
                
        return []
    
    def obtener_puntos_cercanos(ciclovias, punto):
        secuencia = punto['secuencia']

        # Crear la lista de secuencias cercanas
        secuencias_cercanas = [i for i in range(secuencia - 3, secuencia + 4) if i > 0 and i <= len(ciclovias)]

        # Filtrar las filas que cumplen la condición
        return ciclovias[ciclovias['secuencia'].isin(secuencias_cercanas)] 


    df_ciclovia_cercana = []
    segmento_ciclovia = []
    for idx, punto in df_viaje[['latitude', 'longitude']].iterrows():
        #print(f"Progreso puntos ciclovia: {idx}", end="\r")
        if len(df_ciclovia_cercana) == 0: # Buscar ciclovia cercana
            punto_ciclovia_cercana = buscar_punto_ciclovia_cercana(punto, df_ciclovias, umbral_distancia) # Obtener el punto mas cercano de la ciclovia al punto del viaje actual
            if len(punto_ciclovia_cercana) > 0:
                segmento_ciclovia.append(punto_ciclovia_cercana) # Agrega el punto a la lista de puntos del segmento en los que ingreso
                df_ciclovias = df_ciclovias[df_ciclovias['id'] == punto_ciclovia_cercana['id']] # Obtener df de la ciclovia mas cercana
                df_ciclovia_cercana = obtener_puntos_cercanos(df_ciclovias, punto_ciclovia_cercana) # Obtener ventana de ciclovia cercana
            
        else: # Si hay df hay ciclovia cercana
            punto_ciclovia_cercana_aux = buscar_punto_ciclovia_cercana(punto, df_ciclovia_cercana, umbral_distancia) # Punto mas cercano de la ciclovia con ventana
            if len(punto_ciclovia_cercana_aux) > 0: 
                segmento_ciclovia.append(punto_ciclovia_cercana_aux)
                df_ciclovia_cercana = obtener_puntos_cercanos(df_ciclovias, punto_ciclovia_cercana)
            else:
                df_ciclovia_cercana = df_ciclovias.copy() # Si no encuentra punto en la ventana entonces ahhora ya no es ventana si no la ciclovia completa

        if len(segmento_ciclovia) > num_puntos:
            return True

    return False

In [5]:
# Obtener los puntos de las ciclovias
cur.execute(f"SELECT id, latitud, longitud, secuencia FROM ciclovias_cuenca order by id, secuencia;")
rows = cur.fetchall()
puntos_ciclovias = rows.copy()

df_ciclovias = pd.DataFrame(puntos_ciclovias, columns=[desc[0] for desc in cur.description])

In [7]:
# Obtener los segmentos que no son caminata ni viajes en automovil
cur.execute(f'''SELECT DISTINCT s.tripid, s.segid, s.uid 
                FROM public_new.segmentos_viajes_{mes}{año} AS s 
                LEFT JOIN public_new.reglas_segmentos_automovil_{mes}{año} AS au 
                    ON s.tripid = au.tripid AND s.segid = au.segid AND s.uid = au.uid
                LEFT JOIN public_new.reglas_segmentos_caminata_{mes}{año} AS ca 
                    ON s.tripid = ca.tripid AND s.segid = ca.segid AND s.uid = ca.uid
                WHERE au.tripid IS NULL AND ca.tripid IS NULL;
            ''')
rows = cur.fetchall()
id_segmentos = rows.copy()
len(id_segmentos)

33391

In [8]:
# Crear tabla para insertar ids segmentos en bici
consulta_sql = f'''DROP TABLE IF EXISTS reglas_segmentos_bicicleta_{mes}{año};
                    CREATE TABLE IF NOT EXISTS reglas_segmentos_bicicleta_{mes}{año}
                        (
                            tripid integer,
                            segid integer,
                            uid character varying(50)
                        );
                    '''

'''cur.execute(consulta_sql)
conn.commit()'''

In [ ]:
quartiles_graficos_dispercion_manuales = '''
CAMINATA
||---------------------------------------||
Percentiles de velocidad CAMINATA
Percentil 25: 0.842585160645386
Percentil 50 (Mediana): 1.1289679737973142
Percentil 75: 1.3708547759877616
|---------------------------------------|
BICICLETA
||---------------------------------------||
Percentiles de velocidad BICICLETA
Percentil 25: 3.79845044551606
Percentil 50 (Mediana): 4.913760785107003
Percentil 75: 6.261856501510167
|---------------------------------------|
Percentiles de aceleracion BICICLETA
Percentil 25: -0.100823952742842
Percentil 50 (Mediana): -0.0111123992753146
Percentil 75: 0.11854342409689711
|---------------------------------------|
BICICLETA KML
||---------------------------------------||
Percentiles de velocidad BICICLETA
Percentil 25: 2.217772756343824
Percentil 50 (Mediana): 2.94354380175423
Percentil 75: 4.705099565156795
|---------------------------------------|
Percentiles de aceleracion BICICLETA
Percentil 25: -0.06230931025992603
Percentil 50 (Mediana): 0.0014872358895903215
Percentil 75: 0.08298142189818243
||---------------------------------------||
Varianza BICICLETA 
Velocidad: 3.282338004304799
Aceleracion: 0.05749102202483876
Bearing: 10743.477215309482
|---------------------------------------|
Media Velocidad BICICLETA: 3.593528039767074
Media Aceleracón BICICLETA: 0.019987612156943786
Media Tiempos_espera BICICLETA: 14.555555555555555
|---------------------------------------|
BUS
||---------------------------------------||
Percentiles de velocidad BUS
Percentil 25: 2.6072038690586012
Percentil 50 (Mediana): 6.781524691139526
Percentil 75: 10.031165217268553
|---------------------------------------|
Percentiles de aceleracion BUS
Percentil 25: -0.1462496002882079
Percentil 50 (Mediana): 0.08857072967790053
Percentil 75: 0.6974918197345074
|---------------------------------------|
AUTOMOVIL
||---------------------------------------||
Percentiles de velocidad AUTOMOVIL
Percentil 25: 7.666133965789363
Percentil 50 (Mediana): 11.915472229190119
Percentil 75: 17.477035965974075
|---------------------------------------|
Percentiles de aceleracion AUTOMOVIL
Percentil 25: -0.27841678959681654
Percentil 50 (Mediana): 0.032392867837905556
Percentil 75: 0.4045349690399508
||---------------------------------------||
'''

In [9]:
# ---- Buscar segmentos bicicleta
print(f"n_viajes: {len(id_segmentos)}")

prog_anterior = 1 # En uno aun no precesa nada
id_segmentos = id_segmentos[prog_anterior - 1:]
prog = prog_anterior - 1
for id_segmento in id_segmentos:
    prog += 1
    print(f"Progreso: {prog}, Viaje: {id_segmento}", end="\r")

    # Comprobar que segmento no ha sido detectado como viaje bici
    cur.execute(f"SELECT * FROM reglas_segmentos_bicicleta_{mes}{año} where tripid='{id_segmento[0]}' and segid='{id_segmento[1]}' and uid='{id_segmento[2]}'")
    if len(cur.fetchall()) == 0: # Si segmento no esta en viaje bici para no volver a procesar al menos los viajes que ya se sabe que son bici
      # Obtener los puntos del viaje
      cur.execute(f"SELECT * FROM segmentos_viajes_{mes}{año} where tripid='{id_segmento[0]}' and segid='{id_segmento[1]}' and uid='{id_segmento[2]}' order by recorded_at")
      rows = cur.fetchall()
      puntos_viaje = rows.copy()
      df_segmento = pd.DataFrame(puntos_viaje, columns=[desc[0] for desc in cur.description])
      
      # Reglas para procesar solo posibles candidatos a ser bicicleta BICICLETA
      ## Primera regla quita caminatas
      ## Segunda y tercera regla quiere quitar bus y al quitar bus se quita auto 4.91+6.78/2
      ## Tercera regla media q3 de buses aceleracion 0.69 y bicis q3 0.143
      if (df_segmento['speed'].mean() >= 1.8 and df_segmento['speed'].max() <= 11 and df_segmento['speed'].quantile(0.50) < 6.55 
          and df_segmento['aceleration'].quantile(0.25) > -0.16 and df_segmento['aceleration'].quantile(0.75) < 0.25):

        if detectar_segmento_ciclovia(df_segmento, df_ciclovias, umbral_distancia=30, num_puntos=4): # Si entro en la ciclovia al menos 4 veces
          # Construye la consulta SQL con parámetros
          consulta_sql = """
          INSERT INTO {} VALUES (
              %s, %s, %s
          )
          """.format(f'reglas_segmentos_bicicleta_{mes}{año}')

          # Ejecuta la consulta SQL con los valores de la fila actual
          cur.execute(consulta_sql, id_segmento)
          conn.commit() # Hace commit cuando inserta todo el viaje

n_viajes: 16046


In [12]:
# Obtener los segmentos de usuarios detectados en viajes en bici que cumplen las reglas para bici pero que no pasen por ciclovia
cur.execute(f'''SELECT DISTINCT tripid, segid, uid from reglas_segmentos_bicicleta_{mes}{año};
            ''')
rows = cur.fetchall()
segmentos_bicicleta = rows.copy()
id_usuarios_bicicleta = list(set([segmento_bicicleta[2] for segmento_bicicleta in segmentos_bicicleta]))

prog_u = 0
print(f"n_usuarios: {len(id_usuarios_bicicleta)}")
for id_usuario in id_usuarios_bicicleta:
    prog_u += 1
    print(f"Progreso usuarios: {prog_u}") # , Viaje: {id_segmento}

    # Obtener los puntos del viaje
    cur.execute(f"SELECT DISTINCT tripid, segid, uid FROM segmentos_viajes_{mes}{año} where uid='{id_usuario}'")
    rows = cur.fetchall()
    segmentos_usuario_bici = rows.copy() # Pueden incluir los segmentos ya clasificados como bicicleta

    print(f"n_segmentos_usuario: {len(segmentos_usuario_bici)}")
    prog_s = 0
    for id_segmento in segmentos_usuario_bici:
        prog_s += 1
        print(f"Progreso segmentos: {prog_s}, Segmento:  {id_segmento}", end="\r")

        if id_segmento in segmentos_bicicleta: # Si ya esta el segmento en segmentos bici
            continue

        # Obtener los puntos del segmento
        cur.execute(f"SELECT * FROM segmentos_viajes_{mes}{año} where tripid='{id_segmento[0]}' and segid='{id_segmento[1]}' and uid='{id_usuario[0]}' order by recorded_at")
        rows = cur.fetchall()
        puntos_viaje = rows.copy()
        df_segmento = pd.DataFrame(puntos_viaje, columns=[desc[0] for desc in cur.description])
        
        # Si es un usuario que fue detectado que viaja en bicicleta es muy probable que si cumple las velocidades uno de sus segmentos vaya en bici de nuevo
        if (df_segmento['speed'].mean() >= 2 and df_segmento['speed'].max() <= 11 and df_segmento['speed'].quantile(0.50) < 3 
            and df_segmento['aceleration'].quantile(0.25) < -0.07 and df_segmento['aceleration'].quantile(0.75) < 0.1): 
            
            # Construye la consulta SQL con parámetros
            consulta_sql = """
            INSERT INTO {} VALUES (
                %s, %s, %s
            )
            """.format(f'reglas_segmentos_bicicleta_{mes}{año}')

            # Ejecuta la consulta SQL con los valores de la fila actual
            cur.execute(consulta_sql, id_segmento + id_usuario)
            conn.commit() # Hace commit cuando inserta todo el viaje

    # Borrar la salida actual
    clear_output(wait=True)

Progreso usuarios: 42
n_segmentos_usuario: 98


In [10]:
# ---- CLASIFICAR MODO AUTOMOVIL Y BUS

In [8]:
# Conectar a la base de datos PostgreSQL
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
# Crear un cursor para ejecutar consultas
cur = conn.cursor()

In [9]:
# Obtener las rutas de buses
cur.execute(f"SELECT id_linea, linea, latitud_y as latitud, longitud_x as longitud, sentido, secuencia FROM rutas_bus_cuenca;")
rows = cur.fetchall()
puntos_ciclovias = rows.copy()

df_rutas_buses = pd.DataFrame(puntos_ciclovias, columns=[desc[0] for desc in cur.description])

In [ ]:
# --- AUTOMOVIL

In [9]:
# Crear tabla para insertar datos corregidos
consulta_sql = f'''DROP TABLE IF EXISTS reglas_segmentos_automovil_{mes}{año};
                CREATE TABLE IF NOT EXISTS reglas_segmentos_automovil_{mes}{año}
                    (
                        tripid integer,
                        segid integer,
                        uid character varying(50)
                    );
                '''

'''cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()'''

'cur.execute(consulta_sql)\n# Confirma los cambios en la base de datos\nconn.commit()'

In [10]:
cur.execute(f'''SELECT DISTINCT s.tripid, s.segid, s.uid 
                FROM public_new.segmentos_viajes_{mes}{año} AS s 
                LEFT JOIN public_new.reglas_segmentos_bicicleta_{mes}{año} AS bi 
                    ON s.tripid = bi.tripid AND s.segid = bi.segid AND s.uid = bi.uid
                LEFT JOIN public_new.reglas_segmentos_caminata_{mes}{año} AS ca 
                    ON s.tripid = ca.tripid AND s.segid = ca.segid AND s.uid = ca.uid
                WHERE bi.tripid IS NULL AND ca.tripid IS NULL;
            ''')

# Obtener los resultados de la consulta
rows = cur.fetchall()
id_segmentos = rows.copy()
print(len(id_segmentos))

33250


In [11]:
def detectar_parada_bus_trayecto(df_viaje, df_rutas_buses, umbral_distancia):
    def buscar_lineas_cercanas(punto, rutas_buses, umbral_distancia):

        # Calcular la distancia entre el punto de referencia y cada parada en el DataFrame
        rutas_buses.loc[:, 'distancia'] = rutas_buses.apply(lambda row: geodesic((row['latitud'], row['longitud']), (punto['latitude'], punto['longitude'])).meters, axis=1)

        # Filtrar las líneas de autobús que tienen una parada a una distancia máxima dado umbral
        lineas_cercanas = rutas_buses[rutas_buses['distancia'] <= umbral_distancia]
        if len(lineas_cercanas) == 0:
            return False
        return True # Hay parada cercana a punto

    duracion = (df_viaje['recorded_at'].iloc[-1] - df_viaje['recorded_at'].iloc[0]).total_seconds()
    cont_pts = 0
    for idx, punto in df_viaje.iterrows():
        cont_pts += 1
        if idx % 2 != 0: # Solo puntos impares
            # Busca las lineas que pasan por el primer punto del viaje
            if buscar_lineas_cercanas(punto, df_rutas_buses, umbral_distancia):
                return True # Hay punto cercano a parada
            elif duracion > 300 and cont_pts > len(df_viaje)/2: # Si dura mas de 5 minutos y en la mitad del viaje no se ha encontrado paradas
                return False
        
    return False # No hay punto cercano a parada

In [14]:
# ---- Buscar segmentos de automovil, que tengan velocidades de carro o bus pero que no pasen por una parada de bus
def buscar_segmentos_automovil():
    tabla_destino = f'reglas_segmentos_automovil_{mes}{año}'

    print(f"n_segmentos: {len(id_segmentos)}")
    prog_anterior = 1 # En uno aun no precesa nada
    id_segmentos = id_segmentos[prog_anterior - 1:]
    prog = prog_anterior - 1
    for id_segmento in id_segmentos:
        prog += 1
        print(f"Progreso: {prog}", end="\r")   

        # Obtener los ids de todos los segmentos de transporte
        cur.execute(f"SELECT * FROM segmentos_viajes_{mes}{año} where tripid='{id_segmento[0]}' and segid = '{id_segmento[1]}' and uid='{id_segmento[2]}' order by recorded_at")
        rows = cur.fetchall()
        puntos_viaje = rows.copy()
        df_segmento = pd.DataFrame(puntos_viaje, columns=[desc[0] for desc in cur.description])

        #Reglas bus
        '''if df_segmento['speed'].mean() > 5.2 and df_segmento['speed'].max() > 11: 
            segmento_bus = detectar_segmento_bus(df_segmento, df_rutas_buses.copy(), umbral_distancia=25, num_paradas=7)''' 
        # Reglas para segmento carro, salen ed despues de la segmentacion script 3 graficos
        if ((df_segmento['speed'].mean() > 5.2 and df_segmento['speed'].max() > 11) # Posibles viajes en bus
            or (df_segmento['speed'].quantile(0.75) > 13.5 and df_segmento['speed'].quantile(0.25) > 6.5)):

            if detectar_parada_bus_trayecto(df_segmento, df_rutas_buses, umbral_distancia=25): # Si hay parada de bus en el trayecto quitamos no queremos confundir con bus
                continue

            # Construye la consulta SQL con parámetros
            consulta_sql = """
            INSERT INTO {} VALUES (
                %s, %s, %s
            )
            """.format(tabla_destino)

            # Ejecuta la consulta SQL con los valores de la fila actual
            cur.execute(consulta_sql, id_segmento)
            conn.commit()

#buscar_segmentos_automovil()

In [ ]:
# --- BUS

In [10]:
def detectar_segmento_bus(df_viaje, df_rutas_buses, umbral_distancia, num_paradas):

    def buscar_lineas_cercanas(punto, rutas_buses, umbral_distancia):

        # Calcular la distancia entre el punto de referencia y cada parada en el DataFrame
        rutas_buses.loc[:, 'distancia'] = rutas_buses.apply(lambda row: geodesic((row['latitud'], row['longitud']), (punto['latitude'], punto['longitude'])).meters, axis=1)

        # Filtrar las líneas de autobús que tienen una parada a una distancia máxima de 50 metros del punto de referencia
        lineas_cercanas = rutas_buses[rutas_buses['distancia'] <= umbral_distancia]
        if len(lineas_cercanas) == 0:
            return {}
        # Encontrar el índice del elemento más cercano por línea de autobús
        indices_cercanos = lineas_cercanas.groupby('id_linea')['distancia'].idxmin()
        # Seleccionar los elementos más cercanos por línea
        df_lineas_cercanas =  lineas_cercanas.loc[indices_cercanos][['id_linea', 'linea', 'latitud', 'longitud', 'sentido', 'secuencia']]

        # Iterar sobre las filas del DataFrame
        lineas_dict = {}
        for index, row in df_lineas_cercanas.iterrows():
            id_linea = row['id_linea']
            # Verificar si el id_linea ya está en el diccionario
            if id_linea in lineas_dict:
                # Si ya está en el diccionario, agregar la fila al valor correspondiente
                lineas_dict[id_linea].append(row)
            else:
                # Si no está en el diccionario, crear una nueva lista con la fila y asignarla al id_linea
                lineas_dict[id_linea] = [row]

        return lineas_dict

    def obtener_paradas_cercanas(rutas_buses, ultima_parada, umbral_distancia):

        # Calcular la distancia entre el punto de referencia y cada parada en el DataFrame
        rutas_buses.loc[:, 'distancia'] = rutas_buses.apply(lambda row: geodesic((row['latitud'], row['longitud']), (ultima_parada['latitud'], ultima_parada['longitud'])).meters, axis=1)

        # Filtrar las líneas de autobús que tienen una parada a una distancia máxima de 50 metros del punto de referencia
        paradas_cercanas = rutas_buses[rutas_buses['distancia'] <= umbral_distancia]
        
        return paradas_cercanas

    # Reducir puntos para procesar mas rapido
    #df_viaje = eliminar_puntos_cercanos(df_viaje, umbral_distancia - 1)

    # Busca las lineas que pasan por el primer punto del viaje
    dict_lineas_cercanas = buscar_lineas_cercanas(df_viaje.iloc[0], df_rutas_buses, umbral_distancia*2)

    if len(dict_lineas_cercanas) == 0:
        return False
    else:
        # Eliminar lineas de bus en las que no pudo haberse subido
        lineas_parada_inicial = dict_lineas_cercanas.keys()
        df_filtrado = df_rutas_buses[df_rutas_buses['id_linea'].isin(lineas_parada_inicial)]

        parada_punto_final = buscar_lineas_cercanas(df_viaje.iloc[-1], df_filtrado, umbral_distancia*2)
        if len(parada_punto_final) == 0:
            return False
        '''elif len(df_viaje) > 20: # Desde aqui se modifica para detectar qu inicie y termine en parada nada mas
            return True
        else:
            return False'''


        lineas_parada_final = parada_punto_final.keys()
        df_filtrado = df_filtrado[df_filtrado['id_linea'].isin(lineas_parada_final)]
        
        # Obtener solo paradas cercanas a la ultima parada encontrada para no buuscar en todo el dataset
        ultima_parada = list(dict_lineas_cercanas.values())[0][0]
        df_paradas_cercanas = obtener_paradas_cercanas(df_filtrado, ultima_parada,  1500)
        for idx, punto in df_viaje.iterrows():
            if idx > 0: # Ya no el primer punto del viaje

                # Hay que mandar solo el ultimo punto encontrado y el anterior y el siguiente de ambos sentidos
                dict_paradas_cercanas_punto = buscar_lineas_cercanas(punto, df_paradas_cercanas, umbral_distancia) # Puede volver vacio

                if len(dict_paradas_cercanas_punto) > 0: # Si ha parada cercana a punto del viaje
                    # Se actuualiza uultima parada
                    ultima_parada = list(dict_paradas_cercanas_punto.values())[0][0]
                    df_paradas_cercanas = obtener_paradas_cercanas(df_filtrado, ultima_parada,  1500)

                    # Unir dict_paradas_cercanas_punto a dict_lineas_cercanas
                    for clave, valor in dict_paradas_cercanas_punto.items(): # Va a haber un elementto por clave (una parada por id_linea)

                        if clave in dict_lineas_cercanas: # Si la clave ya está en el primer diccionario, agregar los valores a la lista existente
                            
                            lista_existente = dict_lineas_cercanas[clave]
                            for elemento in valor: # Va solo a recorrer 1
                                # Si la parada encontrada para una linea es una parada que aun no estaba 
                                if (elemento['latitud'], elemento['longitud']) not in [(parada['latitud'], parada['longitud']) for parada in lista_existente]:
                                    lista_existente.append(elemento)

                    # Eliminar las lineas que se quedan con menos paradas
                    # Encuentra el máximo número de elementos entre todas las claves
                    maximo_elementos = max(len(valores) for valores in dict_lineas_cercanas.values())
                    # Crea un nuevo diccionario que contenga solo las claves con el máximo número de elementos
                    dict_lineas_cercanas = {clave: valores for clave, valores in dict_lineas_cercanas.items() if len(valores) == maximo_elementos}
                    # Actualizar dataframe para quitar lineas las cuales fuueron eliminadas del diccionario
                    lineas_parada_diccionario = dict_lineas_cercanas.keys()
                    df_filtrado = df_filtrado[df_filtrado['id_linea'].isin(lineas_parada_diccionario)]
                    df_paradas_cercanas = df_paradas_cercanas[df_paradas_cercanas['id_linea'].isin(lineas_parada_diccionario)]
            
            if max(len(valores) for valores in dict_lineas_cercanas.values()) >= num_paradas:
                return True
            
        return False       

In [11]:
# Crear tabla para insertar datos corregidos
consulta_sql = f'''DROP TABLE IF EXISTS reglas_segmentos_bus_{mes}{año};
                    CREATE TABLE IF NOT EXISTS reglas_segmentos_bus_{mes}{año}
                        (
                            tripid integer,
                            segid integer,
                            uid character varying(50)
                        );
                    '''

'''cur.execute(consulta_sql)
conn.commit()'''

'cur.execute(consulta_sql)\nconn.commit()'

In [12]:
cur.execute(f'''SELECT DISTINCT s.tripid, s.segid, s.uid 
                FROM public_new.segmentos_viajes_{mes}{año} AS s 
                LEFT JOIN public_new.reglas_segmentos_automovil_{mes}{año} AS au 
                        ON s.tripid = au.tripid AND s.segid = au.segid AND s.uid = au.uid
                LEFT JOIN public_new.reglas_segmentos_bicicleta_{mes}{año} AS bi 
                    ON s.tripid = bi.tripid AND s.segid = bi.segid AND s.uid = bi.uid
                LEFT JOIN public_new.reglas_segmentos_caminata_{mes}{año} AS ca 
                    ON s.tripid = ca.tripid AND s.segid = ca.segid AND s.uid = ca.uid
                WHERE au.tripid IS NULL AND bi.tripid IS NULL AND ca.tripid IS NULL;
            ''')
# Obtener los resultados de la consulta
rows = cur.fetchall()
id_segmentos = rows.copy()
print(len(id_segmentos))

33248


In [13]:
# ---- Segmentar viajes
tabla_destino = f'reglas_segmentos_bus_{mes}{año}'

print(f"n_segmentos: {len(id_segmentos)}")
prog_anterior = 27071 # En uno aun no precesa nada
id_segmentos = id_segmentos[prog_anterior - 1:]
prog = prog_anterior - 1
for id_segmento in id_segmentos:
    prog += 1
    print(f"Progreso: {prog}", end="\r")   

    # Obtener los ids de todos los segmentos de transporte
    cur.execute(f"SELECT * FROM segmentos_viajes_{mes}{año} where tripid='{id_segmento[0]}' and segid = '{id_segmento[1]}' and uid='{id_segmento[2]}' order by recorded_at")
    rows = cur.fetchall()
    puntos_viaje = rows.copy()
    df_segmento = pd.DataFrame(puntos_viaje, columns=[desc[0] for desc in cur.description]) 

    # Reglas para procesar solo posibles candidatos a ser bicicleta BUS
    if ((df_segmento['speed'].mean() >= 1.8 and df_segmento['speed'].max() <= 11 and df_segmento['speed'].quantile(0.50) < 6.55 
          and df_segmento['aceleration'].quantile(0.25) > -0.16 and df_segmento['aceleration'].quantile(0.75) < 0.25)
        or (df_segmento['speed'].mean() > 5.2 and df_segmento['speed'].max() > 11)): 
        #if df_segmento['speed'].mean() > 5.2 and df_segmento['speed'].max() > 11:

        segmento_bus = detectar_segmento_bus(df_segmento, df_rutas_buses.copy(), umbral_distancia=25, num_paradas=7)

        if ((df_segmento['speed'].mean() >= 1.8 and df_segmento['speed'].max() <= 11 and df_segmento['speed'].quantile(0.50) < 6.55 
          and df_segmento['aceleration'].quantile(0.25) > -0.16 and df_segmento['aceleration'].quantile(0.75) < 0.25)): # Si es posible bicicleta
            if segmento_bus:
                print(f"\n Posible bici es bus? {segmento_bus}")
        
        if not segmento_bus:
            continue
        else:
            #print(id_segmento)
            # Construye la consulta SQL con parámetros
            consulta_sql = """
            INSERT INTO {} VALUES (
                %s, %s, %s
            )
            """.format(tabla_destino)

            # Ejecuta la consulta SQL con los valores de la fila actual
            cur.execute(consulta_sql, id_segmento)
            conn.commit() # Hace commit cuando inserta todo el viaje

n_segmentos: 33248
Progreso: 27195
 Posible bici es bus? False
Progreso: 27479
 Posible bici es bus? False
Progreso: 27501
 Posible bici es bus? False
Progreso: 27575
 Posible bici es bus? False
Progreso: 27751
 Posible bici es bus? False
Progreso: 27798
 Posible bici es bus? False
Progreso: 28050
 Posible bici es bus? False
Progreso: 28389
 Posible bici es bus? False
Progreso: 28889
 Posible bici es bus? False
Progreso: 28945
 Posible bici es bus? False
Progreso: 29072
 Posible bici es bus? False
Progreso: 29238
 Posible bici es bus? False
Progreso: 29271
 Posible bici es bus? False
Progreso: 29284
 Posible bici es bus? False
Progreso: 29294
 Posible bici es bus? False
Progreso: 29301
 Posible bici es bus? False
Progreso: 29312
 Posible bici es bus? False
Progreso: 29435
 Posible bici es bus? False
Progreso: 29527
 Posible bici es bus? False
Progreso: 30080
 Posible bici es bus? False
Progreso: 30081
 Posible bici es bus? False
Progreso: 30082
 Posible bici es bus? False
Progreso: 301